In [1]:
from dataloader import load_data_one_hot_encoded
from trainer import train_model, evaluate_model
from snn import SNNModelSimple
import torch
import matplotlib.pyplot as plt
import pickle

In [ ]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"

train_losses_wd = {}
train_accuracies_wd = {}
val_losses_wd = {}
val_accuracies_wd = {}
test_accuracies_wd = {}

for wd in [0, 1e-5, 1e-4, 5e-4, 1e-3, 5e-3]:
    print(f"Training for weight decay={wd}")
    train_loader, val_loader, test_loader, num_outputs, num_features = load_data_one_hot_encoded(root_folder, file_name, num_steps=100, batch_size=128)

    model = SNNModelSimple(num_features, num_outputs)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=wd)

    train_losses, train_accuracies, val_losses, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10)
    train_losses_wd[wd] = train_losses
    train_accuracies_wd[wd] = train_accuracies
    val_losses_wd[wd] = val_losses
    val_accuracies_wd[wd] = val_accuracies

    test_accuracies_wd[wd] = evaluate_model(model, test_loader, encoding='one-hot')

out_file_name = 'exp4_mse_wd_0_5e-3_t_100_bs_128_ep_10_lr_1e-3.pkl'
with open(out_file_name, 'wb') as f:
    pickle.dump({"train_losses": train_losses_wd, "train_accuracies": train_accuracies_wd, "val_losses": val_losses_wd, "val_accuracies": val_accuracies_wd, "test_accuracies": test_accuracies_wd}, f)
